<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/Graph%20simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture --no-stderr
%pip install langgraph -qU

## Estado

En primer lugar, defina el Estado del grafo.

El esquema de Estado sirve como esquema de entrada para todos los Nodos y Aristas del grafo.


`TypedDict` es una clase que permite definir diccionarios con tipos específicos para sus claves. Es útil para proporcionar anotaciones de tipo más precisas para diccionarios en Python. `TypedDict` está disponible en la biblioteca estándar de Python a partir de la versión 3.8.

`TypedDict`pertenece al módulo `typing` en Python. Es un módulo de la biblioteca estándar que proporciona soporte para anotaciones de tipo (type hints). Estas anotaciones permiten especificar los tipos esperados de variables, argumentos de funciones, valores de retorno y estructuras de datos, lo que mejora la claridad del código y permite a herramientas como **mypy** realizar verificaciones estáticas de tipos.

- **`State`**: Es una clase que hereda de `TypedDict`. Esto significa que `State` define la estructura de un diccionario con claves y tipos de valores específicos.
    
- **`graph_state: str`**: Indica que el diccionario debe tener una clave llamada `graph_state`, y su valor debe ser de tipo `str` (cadena de texto).
    

En resumen, `State` es un diccionario tipado que debe tener la siguiente estructura:

```
{
    "graph_state": "algun_valor_de_tipo_str"
}
```



In [ ]:
from typing import TypedDict

class State(TypedDict):
    graph_state: str

## Nodos

Los nodos no son más que funciones python.

El primer argumento posicional es el estado, como se ha definido anteriormente.

Dado que el estado es un TypedDict con el esquema definido anteriormente, cada nodo puede acceder a la clave, graph_state, con `state['graph_state']`.

Cada nodo devuelve un nuevo valor de la clave de estado graph_state.

Por defecto, el nuevo valor devuelto por cada nodo sobreescribirá el valor de estado anterior.



In [ ]:
def nodo_1(state):
    print("---Nodo 1---")
    return {"graph_state": state['graph_state'] +" Soy "}   #Estamos actualizando el estado (estado anterior + un string) y devolviendolo

def nodo_2(state):
    print("---Nodo 2---")
    return {"graph_state": state['graph_state'] +" feliz!"}

def nodo_3(state):
    print("---Nodo 3---")
    return {"graph_state": state['graph_state'] +" triste!"}

## Aristas

In [ ]:
Las aristas conectan los nodos.

Las aristas normales se utilizan si desea ir siempre, por ejemplo, del nodo_1 al nodo_2.

Las aristas condicionales se utilizan si desea establecer una ruta opcional entre nodos.

Las aristas condicionales se implementan como funciones que devuelven el siguiente nodo a visitar según cierta lógica.

In [ ]:
import random
from typing import Literal


# Esta funcion "decide" / devuelve nodo_2 o nodo_3 de forma aleatoria
def decide_mood(state) -> Literal["nodo_2", "nodo_3"]:

    # Often, we will use state to decide on the next node to visit
    user_input = state['graph_state']

    # Here, let's just do a 50 / 50 split between nodes 2, 3
    if random.random() < 0.5:

        # 50% of the time, we return Node 2
        return "nodo_2"

    # 50% of the time, we return Node 3
    return "nodo_3"

Ahora, construimos el gráfico a partir de los componentes definidos anteriormente.

La clase StateGraph es la clase de gráfico que podemos utilizar.

Primero, inicializamos un StateGraph con la clase State que definimos anteriormente.

Luego, agregamos nuestros nodos y aristas.

Usamos el nodo START, un nodo especial que envía la entrada del usuario al gráfico, para indicar dónde comenzar nuestro gráfico.

El nodo END es un nodo especial que representa un nodo terminal.

Por último, compilamos nuestro gráfico para realizar algunas comprobaciones básicas en la estructura del gráfico.

Podemos visualizar el gráfico mermaid.
Usamos la sintaxis :
graph.add_node(name, value).

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

# Build graph
builder = StateGraph(State)
builder.add_node("nodo_1", nodo_1)
builder.add_node("nodo_2", nodo_2)
builder.add_node("nodo_3", nodo_3)

# Logic
builder.add_edge(START, "nodo_1")
builder.add_conditional_edges("nodo_1", decide_mood)
builder.add_edge("nodo_2", END)
builder.add_edge("nodo_3", END)

# Add
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

## Invocacion del grafo

El gráfico compilado implementa el protocolo Runnable.

Esto proporciona una forma estándar de ejecutar componentes LangChain.

La invocación es uno de los métodos estándar en esta interfaz.

La entrada es un diccionario {"graph_state": "Hola, soy lance."}, que establece el valor inicial para nuestro diccionario de estado del gráfico.

Cuando se llama a la invocación, el gráfico comienza la ejecución desde el nodo START.

Avanza a través de los nodos definidos (nodo_1, nodo_2, nodo_3) en orden.

La arista condicional atravesará desde el nodo 1 al nodo 2 o 3 utilizando una regla de decisión 50/50.

Cada función de nodo recibe el estado actual y devuelve un nuevo valor, que anula el estado del gráfico.

La ejecución continúa hasta que llega al nodo END.

In [ ]:
graph.invoke({"graph_state" : "Hi, this is Lance."})

Reto

Crear un grafo de este tipo y ejecutarlo
Debe tener 8 nodos (donde los nodos 2 y 3 presentan senda opciones) y todos teminan en END



In [ ]:
# Por claridad, volvemos a realizar aqui las importaciones necesarias
import random
from typing import Literal, TypedDict
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

# Definimos la estructura del estado usando TypedDict
class State(TypedDict):
    graph_state: str
    nodo_actual: str

# DEFINIMOS LOS NODOS
def nodo_1(state: State) -> State:
    print("Nodo1—",end="")
    return {"graph_state": state['graph_state'] + "Nodo1—", "nodo_actual": "nodo_1"}

def nodo_2(state: State) -> State:
    print("—Nodo2—",end="")
    return {"graph_state": state['graph_state'] + "—Nodo2—", "nodo_actual": "nodo_2"}

def nodo_3(state: State) -> State:
    print("—Nodo3—",end="")
    return {"graph_state": state['graph_state'] + "—Nodo3—", "nodo_actual": "nodo_3"}

def nodo_4(state: State) -> State:
    print("—Nodo4—",end="")
    return {"graph_state": state['graph_state'] + "—Nodo4—", "nodo_actual": "nodo_4"}

def nodo_5(state: State) -> State:
    print("—Nodo5—",end="")
    return {"graph_state": state['graph_state'] + "—Nodo5—", "nodo_actual": "nodo_5"}

def nodo_6(state: State) -> State:
    print("—Nodo6—",end="")
    return {"graph_state": state['graph_state'] + "—Nodo6—", "nodo_actual": "nodo_6"}

def nodo_7(state: State) -> State:
    print("—Nodo7—",end="")
    return {"graph_state": state['graph_state'] + "—Nodo7—", "nodo_actual": "nodo_7"}

def nodo_8(state: State) -> State:
    print("—Nodo8",end="")
    return {"graph_state": state['graph_state'] + "—Nodo8", "nodo_actual": "nodo_8"}


# CREAMOS UNA FUNCION GENERICA PARA SALIR DE LOS NODOS CONDICIONALES
def decidir_nodo(state) -> Literal["nodo_2", "nodo_3", "nodo_4", "nodo_5", "nodo_6", "nodo_7"]:
    nodo_actual = state["nodo_actual"]
    if nodo_actual == "nodo_1":
        # Decide aleatoriamente entre nodo_2 y nodo_3 con un 50% de probabilidad
        return random.choice(["nodo_2", "nodo_3"])
    elif nodo_actual == "nodo_2":
        # Decide aleatoriamente entre nodo_4 y nodo_5 con un 50% de probabilidad
        return random.choice(["nodo_4", "nodo_5"])
    elif nodo_actual == "nodo_3":
        # Decide aleatoriamente entre nodo_6 y nodo_7 con un 50% de probabilidad
        return random.choice(["nodo_6", "nodo_7"])
    else:
        # Si el nodo actual no es nodo_1, nodo_2 o nodo_3, devuelve None o un mensaje de error
        return nodo_actual + "Nodo no válido"

# Build graph
builder = StateGraph(State)
builder.add_node("nodo_1", nodo_1)
builder.add_node("nodo_2", nodo_2)
builder.add_node("nodo_3", nodo_3)
builder.add_node("nodo_4", nodo_4)
builder.add_node("nodo_5", nodo_5)
builder.add_node("nodo_6", nodo_6)
builder.add_node("nodo_7", nodo_7)
builder.add_node("nodo_8", nodo_8)

# Logic y aristas
builder.add_edge(START, "nodo_1")
builder.add_conditional_edges("nodo_1", decidir_nodo)
builder.add_conditional_edges("nodo_2", decidir_nodo)
builder.add_conditional_edges("nodo_3", decidir_nodo)
builder.add_edge("nodo_4", "nodo_8")
builder.add_edge("nodo_5", "nodo_8")
builder.add_edge("nodo_6", "nodo_8")
builder.add_edge("nodo_7", "nodo_8")
builder.add_edge("nodo_8", END)

# Add
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

# Ejecutamos el grafo
graph.invoke({"graph_state": "", "nodo_actual": "nodo_1"})

### **Reto 2: Sistema de Decisión con Retroalimentación**

#### Descripción:

Debes crear un grafo que modele un sistema de decisión con retroalimentación. El grafo tendrá los siguientes componentes:

1. **Nodos principales**:
    
    - `nodo_inicio`: El nodo inicial.
        
    - `nodo_A`, `nodo_B`, `nodo_C`: Nodos intermedios que toman decisiones.
        
    - `nodo_final`: El nodo final del grafo.
        
2. **Lógica de decisiones**:
    
    - Desde `nodo_inicio`, el grafo debe decidir aleatoriamente entre `nodo_A`, `nodo_B` o `nodo_C`.
        
    - Desde `nodo_A`, el grafo debe decidir aleatoriamente entre volver a `nodo_inicio` o ir a `nodo_final`.
        
    - Desde `nodo_B`, el grafo debe decidir aleatoriamente entre ir a `nodo_A` o `nodo_C`.
        
    - Desde `nodo_C`, el grafo siempre debe ir a `nodo_final`.
        
3. **Retroalimentación**:
    
    - Si el grafo pasa por `nodo_A` más de una vez, debe registrar cuántas veces ha pasado por este nodo en el estado.
        
    - Si el grafo pasa por `nodo_A` más de 3 veces, debe forzar la salida a `nodo_final`.
        
4. **Salida**:
    
    - El grafo debe terminar en `nodo_final` después de un número razonable de iteraciones.
        

---

### Requisitos del Reto:

1. **Estado del grafo**:
    
    - El estado debe incluir:
        
        - `graph_state`: Una cadena que registre el camino recorrido.
            
        - `nodo_actual`: El nodo en el que se encuentra actualmente.
            
        - `contador_A`: Un contador que registre cuántas veces se ha pasado por `nodo_A`.
            
2. **Funciones de nodo**:
    
    - Cada nodo debe actualizar el estado del grafo y decidir el siguiente nodo según la lógica descrita.
        
3. **Condiciones de retroalimentación**:
    
    - Si el grafo pasa por `nodo_A` más de 3 veces, debe forzar la salida a `nodo_final`.
        
4. **Visualización**:
    
    - Usa `graph.get_graph().draw_mermaid_png()` para visualizar el grafo.

![JJJ](https://github.com/juanfranbrv/curso-langchain/blob/80c30122b64ff14b3d2765f132e0c43d6c3d3310/images/MI__start__.png?raw=true)

<img src="https://github.com/juanfranbrv/curso-langchain/blob/80c30122b64ff14b3d2765f132e0c43d6c3d3310/images/MI__start__.png?raw=true">


In [ ]:
# Por claridad, volvemos a realizar aqui las importaciones necesarias
import random
from typing import Literal, TypedDict
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

# Definimos la estructura del estado usando TypedDict
class State(TypedDict):
    graph_state: str  # cadena que registra el camino recorrido de la forma INICIO-A--B-C-INICIO-A-FINAL
    nodo_actual: str
    contador_A: int

# DEFINIMOS LOS NODOS
def INICIO(state: State) -> State:
    return {"graph_state": state['graph_state'] + "—INICIO—", "nodo_actual": "INICIO", "contador_A": state["contador_A"]}

def nodo_A(state: State) -> State:
    # print("—Nodo2—",end="")
    return {"graph_state": state['graph_state'] + "—A—", "nodo_actual": "nodo_A", "contador_A": state["contador_A"]+1}

def nodo_B(state: State) -> State:
    # print("—Nodo3—",end="")
    return {"graph_state": state['graph_state'] + "—B—", "nodo_actual": "nodo_B", "contador_A": state["contador_A"]}

def nodo_C(state: State) -> State:
    # print("—Nodo4—",end="")
    return {"graph_state": state['graph_state'] + "—C—", "nodo_actual": "nodo_C", "contador_A": state["contador_A"]}

def FINAL(state: State) -> State:
    # print("—Nodo5—",end="")
    return {"graph_state": state['graph_state'] + "—FINAL—", "nodo_actual": "FINAL", "contador_A": state["contador_A"]}


# CREAMOS UNA FUNCION GENERICA PARA SALIR DE LOS NODOS CONDICIONALES
def decidir_nodo(state) -> Literal["INICIO", "nodo_A", "nodo_B","nodo_C","FINAL"]:
    nodo_actual = state["nodo_actual"]
    contador_A= state["contador_A"]
    if nodo_actual == "INICIO":
        # Decide aleatoriamente A,B,C
        return random.choice(["nodo_A", "nodo_B", "nodo_C"])
    elif nodo_actual == "nodo_A":
        # Decide aleatoriamente entre INICIO y FINAL con un 50% de probabilidad, si no ha pasado por aqui >3 veces
        if contador_A>3:
            return "FINAL"
        else:
            return random.choice(["INICIO", "FINAL"])
    elif nodo_actual == "nodo_B":
        # Decide aleatoriamente entre nodo_
        return random.choice(["nodo_A", "nodo_C"])
    else:
        # Si el nodo actual no es nodo_1, nodo_2 o nodo_3, devuelve None o un mensaje de error
        return nodo_actual + "Nodo no válido"

# Build graph
builder = StateGraph(State)
builder.add_node("INICIO", INICIO)
builder.add_node("nodo_A", nodo_A)
builder.add_node("nodo_B", nodo_B)
builder.add_node("nodo_C", nodo_C)
builder.add_node("FINAL", FINAL)

# Logic y aristas
builder.add_edge(START, "INICIO")
builder.add_conditional_edges("INICIO", decidir_nodo)
builder.add_conditional_edges("nodo_A", decidir_nodo)
builder.add_conditional_edges("nodo_B", decidir_nodo)
builder.add_edge("nodo_C", "FINAL")
builder.add_edge("FINAL", END)


# Add
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

# Ejecutamos el grafo
graph.invoke({"graph_state": "", "nodo_actual": "nodo_1", "contador_A": 0})


## Reto 3: Grafo con Ponderación de Caminos y Decisiones Condicionales

#### Descripción

Construye un grafo en LangGraph en el que:

1.  Los nodos representen estaciones en un viaje ficticio por tren.
2.  Cada nodo devuelve el estado del viaje, incluyendo el nombre de la estación y el tiempo acumulado del trayecto.
3.  Las transiciones entre estaciones tengan **pesos aleatorios**, representando el tiempo en minutos para llegar a la siguiente estación.
4.  Los pesos influyan en la decisión del camino a tomar: siempre selecciona el camino con **el menor peso**.
5.  El grafo tiene al menos 6 estaciones (nodos) y debe incluir una estación final.

#### Ejemplo del Flujo

-   Empiezas en la estación `Estación A`.
-   Desde `Estación A`, puedes decidir entre ir a `Estación B` o `Estación C`.
    -   El peso (tiempo) entre A -> B es 10 minutos.
    -   El peso (tiempo) entre A -> C es 15 minutos.
    -   Decides ir a `Estación B` porque es más rápido.
-   El viaje termina en `Estación F`, acumulando el tiempo total del trayecto

#### Detalles Específicos

1.  **Nodos requeridos:**
    
    -   `Estación A`, `Estación B`, `Estación C`, `Estación D`, `Estación E`, `Estación F`.
2.  **Transiciones requeridas:**
    
    -   Al menos 2 opciones posibles en cada nodo (excepto en el nodo final).
3.  **Estado del grafo:**
    
    -   Incluye `current_station` y `total_time` como parte del estado.
4.  **Función de decisión personalizada:**
    
    -   Usa un diccionario para definir pesos entre estaciones y selecciona dinámicamente el mejor camino (menor peso).
5.  **Salida:**
    
    -   Al final, imprime el tiempo total del trayecto y la ruta tomada.

### Pista

-   Usa un diccionario como este para manejar los pesos:


`pesos = {     "Estación A": {"Estación B": 10, "Estación C": 15},     "Estación B": {"Estación D": 20, "Estación E": 25},     "Estación C": {"Estación E": 10, "Estación D": 15},     "Estación D": {"Estación F": 30},     "Estación E": {"Estación F": 20} }`

-   En la función de decisión, usa `min(pesos[nodo_actual], key=pesos[nodo_actual].get)` para encontrar la estación más cercana.


👉🏻En el contexto de LangGraph, el estado debe actualizarse SOLO dentro de las funciones de los nodos siempre que quieras que los cambios sean persistentes y pasados al siguiente nodo. Esto se debe a cómo LangGraph maneja el estado

Razones para Actualizar el Estado en los Nodos
Estado Inmutable:

El estado que recibe cada función de nodo es efectivamente una copia. Si intentas modificarlo directamente (por ejemplo, state["clave"] += valor), ese cambio no se reflejará en el flujo del grafo porque LangGraph espera que cada función retorne un nuevo estado actualizado.
Debe actualizarse mediante el retorno, nunca modificarlo directamente.

In [ ]:
from typing import Literal, TypedDict
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

# Definimos la estructura del estado
class State(TypedDict):
    estacion_actual: str
    total_acumulado: int
    ruta: str

# DEFINIMOS LOS NODOS
def estacion_A(state: State) -> State:
    print("—Estación A—", end="")
    return {
        "estacion_actual": "Estación A",
        "total_acumulado": state["total_acumulado"],
        "ruta": state["ruta"] + "—A—",
    }

def estacion_B(state: State) -> State:
    print("—Estación B—", end="")
    tiempo = pesos[state["estacion_actual"]]["Estación B"]
    return {
        "estacion_actual": "Estación B",
        "total_acumulado": state["total_acumulado"] + tiempo,
        "ruta": state["ruta"] + "—B—",
    }

def estacion_C(state: State) -> State:
    print("—Estación C—", end="")
    tiempo = pesos[state["estacion_actual"]]["Estación C"]
    return {
        "estacion_actual": "Estación C",
        "total_acumulado": state["total_acumulado"] + tiempo,
        "ruta": state["ruta"] + "—C—",
    }

def estacion_D(state: State) -> State:
    print("—Estación D—", end="")
    tiempo = pesos[state["estacion_actual"]]["Estación D"]
    return {
        "estacion_actual": "Estación D",
        "total_acumulado": state["total_acumulado"] + tiempo,
        "ruta": state["ruta"] + "—D—",
    }

def estacion_E(state: State) -> State:
    print("—Estación E—", end="")
    tiempo = pesos[state["estacion_actual"]]["Estación E"]
    return {
        "estacion_actual": "Estación E",
        "total_acumulado": state["total_acumulado"] + tiempo,
        "ruta": state["ruta"] + "—E—",
    }

def estacion_F(state: State) -> State:
    print("—Estación F—", end="")
    tiempo = pesos[state["estacion_actual"]]["Estación F"]
    return {
        "estacion_actual": "Estación F",
        "total_acumulado": state["total_acumulado"] + tiempo,
        "ruta": state["ruta"] + "—F—",
    }

# Pesos entre estaciones
pesos = {
    "Estación A": {"Estación B": 10, "Estación C": 15},
    "Estación B": {"Estación D": 20, "Estación E": 25},
    "Estación C": {"Estación E": 10, "Estación D": 15},
    "Estación D": {"Estación F": 30},
    "Estación E": {"Estación F": 20},
}

# Función para decidir la siguiente estación
def decidir_siguiente_estacion(state: State) -> Literal["Estación B", "Estación C", "Estación D", "Estación E", "Estación F"]:
    estacion_actual = state["estacion_actual"]
    if estacion_actual in pesos:
        # Selecciona la estación con el menor peso
        #min() puede encontrar la key con menor valor !!
        siguiente_estacion = min(pesos[estacion_actual], key=pesos[estacion_actual].get)
        return siguiente_estacion
    else:
        return "Estación F"  # Si no hay más opciones, ir a la estación final

# Instanciamos un Grafo
builder = StateGraph(State)

# Añadir nodos
builder.add_node("Estación A", estacion_A)
builder.add_node("Estación B", estacion_B)
builder.add_node("Estación C", estacion_C)
builder.add_node("Estación D", estacion_D)
builder.add_node("Estación E", estacion_E)
builder.add_node("Estación F", estacion_F)

# Añadimos la lógica y las aristas
builder.add_edge(START, "Estación A")
builder.add_conditional_edges("Estación A", decidir_siguiente_estacion)
builder.add_conditional_edges("Estación B", decidir_siguiente_estacion)
builder.add_conditional_edges("Estación C", decidir_siguiente_estacion)
builder.add_conditional_edges("Estación D", decidir_siguiente_estacion)
builder.add_conditional_edges("Estación E", decidir_siguiente_estacion)
builder.add_edge("Estación F", END)

# Compilamos el grafo
graph = builder.compile()

# Estado inicial
initial_state = {"estacion_actual": "Estación A", "total_acumulado": 0, "ruta": ""}

# Ejecutar el grafo
result = graph.invoke(initial_state)

# Mostrar resultados
print("\nRuta tomada:", result["ruta"])
print("Tiempo total del trayecto:", result["total_acumulado"], "minutos")